In [1]:
import  requests 
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib3.exceptions import SSLError

from requests.exceptions import HTTPError
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
import time
import sys

In [2]:

link = 'https://www.aa.com.tr/en/search/?s='
# keyword = 'Turkish Airlines'
keyword = 'akbank'

keyword2 = '+'.join(keyword.split())
keyword3 = '_'.join(keyword.split())

link += keyword2
print(link)


https://www.aa.com.tr/en/search/?s=akbank


In [3]:
chrome_driver_path = 'selenium_p/chromedriver'

In [4]:

def getArticle(link):
    text = ''
    
#     print('-')
    try: 
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        reg_url = link
        req = Request(url=reg_url, headers=headers) 
        html = urlopen(req).read()  

        soup = BeautifulSoup(html, "html.parser")

        date = soup.find('span',{'class':'tarih'}).text
        data = soup.find('div',{'class':'detay-icerik'}).findAll('p')
        for p in data:
            text +=p.text
            
        return [date,text]
        
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
        pass
    except Exception as err:
        print(f'Other1 error occurred: {err}')  
        pass

In [5]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.set_page_load_timeout('10')
driver.get(link)

result = []

news_count = int(driver.find_element_by_id('spnNewsCount').get_attribute('innerHTML'))
for i in range(1,news_count,20):
    driver.find_element_by_link_text('Continue').send_keys(Keys.ENTER)
    time.sleep(2)
    print(i)
    
data = driver.find_element_by_id('myTabContent').find_elements_by_class_name('row')
for news in data:
    
    link = news.find_element_by_tag_name('a').get_attribute('href')
    headline = news.find_element_by_tag_name('h4').get_attribute('innerHTML')
    description = news.find_element_by_tag_name('p').get_attribute('innerHTML')
    date,text = getArticle(link)

    result.append([headline,description,text,date,link])


time.sleep(4)
driver.quit()


   
df = pd.DataFrame(result, columns =['headline','description','article','date','article_link'])
df.to_csv(f'apis_datasets/{keyword3}_AnadoluAgency.csv', encoding='utf-8', index=False)


1
21
41
61
81
101
121
141
161
181
201
221
241


In [6]:
df.head()

,headline,description,article,date,article_link
0,European bank loans $47M to Turkish leasing co...,New funds to enable company to increase availa...,ANKARAThe European Bank for Reconstruction and...,28.07.2020,https://www.aa.com.tr/en/economy/european-bank...
1,European bank’s investments in Turkey near $1B...,European Bank for Reconstruction and Developme...,ISTANBUL The European Bank for Reconstruction ...,22.07.2020,https://www.aa.com.tr/en/economy/european-bank...
2,Turkey's Akbank realizes first eurobond issuance,Bank says it issued first eurobond in Turkish ...,ANKARATurkey-based private lender Akbank annou...,01.07.2020,https://www.aa.com.tr/en/economy/turkeys-akban...
3,European bank loans $75M to Turkey’s Akbank,EBRD says funds will boost Akbank's lending to...,ANKARAThe European Bank for Reconstruction and...,12.06.2020,https://www.aa.com.tr/en/economy/european-bank...
4,Turkish Airlines remains top brand in Turkey,Flag carrier has nearly $2B brand value in 202...,ISTANBULTurkey's flag carrier Turkish Airlines...,09.06.2020,https://www.aa.com.tr/en/turkey/turkish-airlin...
